In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46458
2,Huelva,Confirmados PDIA 14 días,425
3,Huelva,Tasa PDIA 14 días,"82,81855915193796"
4,Huelva,Confirmados PDIA 7 días,209
5,Huelva,Tasa PDIA 7 dias,"40,72724438295302"
6,Huelva,Total Confirmados,46661
7,Huelva,Curados,45317
8,Huelva,Fallecidos,420


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46458.0


/tmp/ipykernel_54990/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12774.0


/tmp/ipykernel_54990/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_54990/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_54990/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_54990/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2247 personas en los últimos 7 días 

Un positivo PCR cada 1199 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46458.0,209.0,425.0,513170.0,40.727244,82.818559,52.0
Huelva-Costa,27468.0,116.0,240.0,289548.0,40.062442,82.887811,34.0
Huelva (capital),12774.0,64.0,120.0,143837.0,44.494810,83.427769,23.0
Condado-Campiña,14449.0,72.0,150.0,156231.0,46.085604,96.011675,13.0
Sierra de Huelva-Andévalo Central,4115.0,21.0,35.0,67391.0,31.161431,51.935718,5.0
Palos de la Frontera,1096.0,22.0,31.0,11742.0,187.361608,264.009538,5.0
Almonte,2346.0,15.0,27.0,24507.0,61.207002,110.172604,3.0
Lepe,3053.0,6.0,10.0,27880.0,21.520803,35.868006,3.0
Isla Cristina,3177.0,14.0,21.0,21393.0,65.441967,98.162950,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,58.0,5.0,13.0,713.0,701.262272,1823.281907,0.0
Almonaster la Real,91.0,5.0,6.0,1788.0,279.642058,335.570470,1.0
Cartaya,2156.0,24.0,59.0,20083.0,119.504058,293.780810,2.0
Palos de la Frontera,1096.0,22.0,31.0,11742.0,187.361608,264.009538,5.0
Villanueva de las Cruces,21.0,1.0,1.0,387.0,258.397933,258.397933,0.0
Minas de Riotinto,189.0,2.0,6.0,3812.0,52.465897,157.397692,0.0
Moguer,2043.0,7.0,31.0,21867.0,32.011707,141.766132,2.0
Almonte,2346.0,15.0,27.0,24507.0,61.207002,110.172604,3.0
Bollullos Par del Condado,1312.0,14.0,15.0,14387.0,97.310072,104.260791,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Ayamonte,2113.0,1.0,8.0,21104.0,4.738438,37.907506,1.0,0.125000
Moguer,2043.0,7.0,31.0,21867.0,32.011707,141.766132,2.0,0.225806
Rociana del Condado,675.0,1.0,4.0,7939.0,12.596045,50.384179,0.0,0.250000
Aljaraque,1649.0,5.0,16.0,21474.0,23.283971,74.508708,2.0,0.312500
Trigueros,546.0,2.0,6.0,7862.0,25.438820,76.316459,1.0,0.333333
Minas de Riotinto,189.0,2.0,6.0,3812.0,52.465897,157.397692,0.0,0.333333
Bonares,468.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
San Juan del Puerto,770.0,3.0,8.0,9411.0,31.877590,85.006907,1.0,0.375000
Campofrío,58.0,5.0,13.0,713.0,701.262272,1823.281907,0.0,0.384615
